# Lab01 - Preprocesamiento de Datos

Antes de empezar a entrenar un modelo es importante **conocer los datos** para alimentar el modelo con datos que aporten valor y nos ayuden a realizar una correcta Predicción/Clasificación/Segmentación.  Esta actividad se conoce como **Análisis Exploratorio (EDA)**  
Vamos a analizar dos atributos importantes a la hora de definir qué variables van a alimentar el modelo:
- Calidad de Datos
- Correlacion entre los Datos

## Configuración
Trate de ejecutar la siguiente celda para validar que las librerías que vamos a utilizar hoy se encentran instaladas correctamente.  Si no es así, instale las librerías faltantes en una línea de comando con el código que se encuentra más abajo.  Luego, vuelva a intentar importarlas hasta que la celda se ejecute sin errores.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

### Comandos de Instalación (Si son necesarios)
<pre><code>pip install pandas</code></pre>
<pre><code>pip install numpy</code></pre>
<pre><code>pip install seaborn</code></pre>
<pre><code>pip install matplotlib</code></pre>

## Cargar el set de datos
Vamos a usar un set de datos del [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) en donde se registran los atributos de varias botellas de vino junto con su calidad.  Este dataset ha sido modificado un poco para trabajar la limpieza y calidad en este laboratorio.  La versión original puede descargrse [aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv)

Vamos a cargar este dataset en este laboratorio para procesarlo y dejarlo listo para los **siguientes laboratorios** de predicción y clasificación.  De esta forma usaremos el mismo dataset a lo largo de todo el flujo del curso como si se tratara de un proyecto real!

In [ ]:
wine_df = pd.read_csv('../data/winequality-white.csv', sep=';')
len(wine_df)

## Calidad de Datos  
Lo primero que debemos hacer, y que parece obvio, es revisar una muestra de los datos para darnos una idea de los valores que vienen en cada columna/atributo

In [ ]:
wine_df.head()

También es importante conocer con anterioridad el tamaño de los datos con los que estamos tratando

In [ ]:
wine_df.shape

El atributo shape nos devuelve una tupla de la forma (filas, columnas).  En este caso, la cantidad de filas (4898) es la cantidad de muestras o registros a analizar y la cantidad de columnas (12) son los atributos o variables de cada muestra.  


Otra buena práctica al iniciar el Análisis Exploratorio es validar los tipos de dato con los que estamos tratando.  Esto nos permitirá tratar cada atributo de la manera correspondiente o realizar las transformaciones necesarias.

In [ ]:
wine_df.info()

En este caso, todas las variables son numéricas (float o int), lo que las hace buenas candidatas para un modelo, por ejemplo de regresión.  Si se encontraran variables categóricas (string) deberíamos hacer la validación y transformación de estos valores.  

Ya con una idea más clara de la estructura de datos con la que estamos tratando, realicemos un análisis matemático rápido sobre los datos:

In [ ]:
wine_df.describe()

Esto nos entrega un resumen de las medidas de tendencia y disperción de los atributos **numéricos** de nuestro dataset:  
- Conteo de registros: Nos permite validar si algunos campos vienen Nulos (NaN)
- Promedio y Desviación estándar: Nos da una idea de la disperción de cada atributo
- Valores Mínimo y Máximo: Nos permite conocer el dominio de valores entre los que se distribuye cada atributo
- Cuartiles: Nos da un resumen de la distribución de los valores y posibles atípicos  

Ya tenemos un poco más clara la **estructura** de los datos, pero con solo ver una muestra de 5 registros, no podemos estar seguros de conocer todos los posibles **valores** de cada columna.  

### Datos Faltantes
En Python, un dato vacío o faltante es identificado con el valor *NaN* (Not a Number) y debemos identificar qué valores faltantes hay en cada atributo y entender su significado.  Ya lo vimos en el conteo de valores de la celda anterior, pero tambien podemos verlo particularmente:

In [ ]:
wine_df.isna().sum()

Hay dos variables que llaman nuestra atención: *citric acid* y *color*. Qué podemos hacer con estos datos?
- Si son datos inválidos podemos eliminarlos: OJO, aunque esto asegura que tendremos solo registros *limpios*, también perderemos registros que podrían contener otra información útil para el modelo.
- Si son datos inválidos también podemos reemplazarlos con valor válido, por ejemplo co el promedio de los valores válidos de ese mismo atributo. De esta forma conservamos la cantidad de registros y no perdemos valor para ese atributo, pero OJO: el valor de reemplazo debe tener sentido para el negocio.
- Si son datos válidos para el negocio, por ejemplo, se acostumbra dejar vacío cuando es valor 0, podemos hacer este reemplazo directamente para que el modelo entienda eso.  OJO: En este caso se deben tener muy claras las reglas de negocio. **Este es el caso de "citric acid"**
- Si son **muchos** los valores en nulo en una columna en partcular, se puede considerar no tener en cuenta ese atributo en el análisis porque no estaría aportando valor diferencial a cada muestra. **Este es el caso de "color"**  

Ya vimos en la descripción de datos que el mínimo valor de *citric acid* es 0.12, cuando es posible quela acidez de un vino tinto sea 0 y no tenemos valores en 0. Coincidencia?  
Vamos a arriesgarnos a reemplazar los valores por cero (0), para no tener que eliminar estos registros

In [ ]:
wine_df.fillna(value={'citric_acid': 0}, inplace=True)

El caso de *color* es distinto, son **muchos** los valores faltantes (3593 de 4898).  Veamos qué valores tienen los pocos registros que sí aportan información

In [ ]:
wine_df.groupby('color')['score'].count()

Además de los 3593 valores en Nulo, se tienen 428 en valor *8* y 877 en valor *13*.
Estos valores realmente no nos dicen nada, si no conocemos el significado de cada uno.  Lo mejor en este caso es eliminar la columna ya que no aportaría nada al modelo, ni es algo que podamos explicar.

In [ ]:
wine_df.drop('color',axis=1, inplace=True)

### Valores Atípicos
Un valor atípico (en inglés *Outlier*) es un valor que está por fuera del dominio normal de una variable.  Por ejemplo, en un set de datos de personas, el dominio de la variable *edad* podría ser entre 1 y 99 años.  Si se encuentra un valor negativo, o un valor muy alto (Por ejemplo, 200) que es imposible, o raro dentro del concepto (edad)es considerado atípico y debe ser tratado con cuidado.  
Ya en celdas alteriores vimos una forma de detectar atípicos de forma general en el dataset con la función *df.describe()*, ya que nos entrega los quartiles.  Sin embargo, siempre es más fácil entender los datos gráficamente:

In [ ]:
pd.DataFrame.hist(wine_df, figsize=(20,20), bins=50);

Esto nos permite visualizar la distribución de cada variable, valores más comúnes y de pronto algunos atípicos.  Pero la mejor forma de determinar si hay datos fuera de lo normal es con un análisis estadístico de cajas:

In [ ]:
plt.subplots(figsize=(15,3))
sns.boxplot(wine_df['pH'], color='orange');

Dicen los [expertos](https://www.aprenderdevino.es/ph-y-vino/), que el pH de un vino tinto varía entre 3.3 y 3.6 g/l.  Éste conocimiento de negocio, así como la gráfica anterior nos permite dudar un poco de las muestras con valores superiores, por ejemplo a **4**.  *Este valor puede variar como consideremos conveniente.  No se recomienda cortar estrictamente en el intervalo que dicta la regla de negocio (3.3 - 3.6) pues algunos atípicos cercanos pueden ser posibles enla naturaleza*

In [ ]:
wine_df=wine_df[wine_df['pH']<4]
plt.subplots(figsize=(15,3))
sns.boxplot(wine_df['pH'], color='orange');

In [ ]:
wine_df.describe()

Realizando de nuevo nuestro resumen de métricas estadísticas, vemos que ya no se tienen valores faltantes es nungun atributo. La distribución de las columnas que hemos modificado ha cambiado un poco, pero su desviación y distribución en general se ajusta más para un análisis predictivo.  

El análisis predictivo en el que vamos a trabajar en los siguientes laboratorios se realizará sobre las siguientes variables objetivo:
- Análisis Predictivo sobre la variable *score* (Archivo winequality-white.csv)
- Análisis de Clasificación sobre la variable *high_quality* (Archivo winequality-red.csv)

Vamos ahora a preparar los datos y las variables orientandolos a este tipo de modelos

### Correlación y Multicolinearidad
Otro análisis que debemos realizar antes de empezar a crear un modelo, es elegir cuales variables incluir y cuales no dependiendo de la variable que vamos a predecir, ésta se conoce como **variable objetivo**.  Es importante tener en cuenta atributos que aporten valor predictivo a la variable objetivo y eliminar de nuestro análisis aquellas que no o que, por el contrario introduscan ruido al modelo (Ya hemos hecho esto, por ejemplo con el atributo *color*).  Esto facilitará al modelo de tener que elegir y buscar tendencias sobre muchas variables que, de anteano sabemos que no están relacionadas con nuestra variable objetivo.  Este proceso se le conoce como **feature engineering**  

Además de eliminar las columnas con ruido o con muchos Nulos, como ya hicimos con el atributo *color*, es importante validar la **correlación entre variables**, es decir, analizar la relación de cambio entre cada pareja de variables, si una varibale aumenta, aumenta la otra? disminuye? En la misma proporción?.  
Cuando dos **variables predictivas** están altamente correlacionadas se le llama [multicolinearidad](https://en.wikipedia.org/wiki/Multicollinearity), y sobre esto debemos tener cuidado, pues puede desestabilizar nuestro modelo.  Entre más correlacionadas estén dos variables, no sirve de nada incluir las dos en el modelo pues menor valor le aportarán, una es redundante con la otra.

Por otro lado, si una variable predictiva está altamente correlacionada con la **variable objetivo**, en este caso es algo bueno! Significa que es una variable que sí querremos incluir en el modelo la que explican en gran parte la varianza de la misma.

Vamos a analizar **visualmente** la relación de las variables entre sí, así como con la variable objetivo *score*:

In [ ]:
# dependiendo de la cantidad de registros, esta función puede tomar unos segundos en ejecutar
pd.plotting.scatter_matrix(wine_df.drop('score',axis=1), 
                           c=wine_df['score'], figsize=(20,20));

Esta función nos permite realizar varios análisis al tiempo:
- Mapas de dispersión de cada pareja de variables: Qué variables se encuentran correlacionadas entre sí? Positivamente o Negativamente?
- En la columna o fila de gráficas de *score* (nuestra variable objetivo para predicción) nota alguna variable predictiva altamente correlacionada que nos aporte alto valor predictivo?
- La diagonal nos muestra la disperción de cada variable: Nota algo raro sobre la dispersión de alguna variable en particular?
- Los colores en la gráficas muestran la variable *score* (nuestra variable objetivo), nota alguna variable predictiva altamente correlacionada que nos aporte alto valor predictivo?

Además de verlo gráficamente, existe un indicador llamado **índice de correlación** que permte calcular numéricamente qué tan relacionadas están las variables. Este indicador varía de -1 (Correlacion *altamente negativa*) a 1 (Correlacion *altamente positiva*)

In [ ]:
wine_df.corr()

Es más preciso que revisar las gráficas una a una, pero ahora son muchos números y decimales lo que hace difícil lectura o la toma de alguna decisión...
Qué pasa si vemos estos valores como un mapa de calor?

In [ ]:
plt.subplots(figsize=(9,7))
sns.heatmap(wine_df.corr(), annot=True, cmap='YlOrRd');

Mucho mejor!  Qué podemos observar ahora de los atributos?  
Hay unos mas correlacionados que otros? Eso es bueno? o malo?  
*Recuerde que la columnas **score** será nuestras variables objetivo, alta correlación positiva o negativa con ellas es algo **bueno**.*  

Tenga en cuenta sus descubrimientos para los próximos laboratorios.  
Por ahora, vamos a guardar el set de datos con las transformaciones y limpieza que hemos realizado para usarlo en las siguientes clases.

In [ ]:
wine_df.to_csv('data/winequality-white_clean.csv', index=False)

**Taller**
- Repita el ejercicio para el archivo *winequality-red.csv* en la misma ruta para corregir la calidad de datos y validar su correlación.  (En ese caso, el archivo no tiene el campo *score* sino *high_quality* que en ese caso será nuestra variable objetivo pues es el archivo que usaremos para los laboratorios de **Clasificación**)
- Qué otros descubrimientos encuentra en los datos? Intente trabajar con estadarización de datos.